In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import h5py
from nd2_to_caiman import np_arr_from_nd2
import labrotation.file_handling as fh
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import matplotlib.colors as mcolors
from math import floor, sqrt, atan2
from datetime import datetime
import json
from labrotation import json_util
import scipy
from scipy import ndimage
import datadoc_util
from statsmodels.nonparametric.smoothers_lowess import lowess
import pandas as pd
import seaborn as sns
import multiprocess as mp  # multiprocessing does not work with IPython. Use fork instead.
import os
import random  # for possible stochastic algorithms

In [ ]:
grid_shape = (8,8)

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")
print(data_docu_folder)

In [ ]:
ddoc = datadoc_util.DataDocumentation(data_docu_folder)
ddoc.loadDataDoc()

In [ ]:
onsets_h5_fpath = fh.open_file("Open extracted onsets and grid h5 file!")
print(onsets_h5_fpath)

In [ ]:
df_onsets = pd.read_hdf(onsets_h5_fpath)

In [ ]:
grid_data1 = df_onsets.pivot_table(index="row", columns="col", values="onset1", aggfunc="median")
grid_data2 = df_onsets.pivot_table(index="row", columns="col", values="onset2", aggfunc="median")
grid_datasz = df_onsets.pivot_table(index="row", columns="col", values="onset_sz", aggfunc="median")

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))
sns.despine(f, left=True, bottom=True)
#sns.scatterplot(x="x", y="y", hue="onset_sz", size="onset_sz",
#                sizes=[5, 40, 80, 160, 240, 320], linewidth=0,
#                data=onsets_df, ax=ax, palette=hues)
sns.scatterplot(x="x", y="y", hue="quantile1", size="onset1",
                sizes=(10,80), linewidth=0,
                data=df_onsets, ax=ax)
ax.invert_yaxis()  # invert to match imshow() and in general, nd2 videos: (0, 0) is top left corner
plt.show()

In [ ]:
df_onsets

In [ ]:
quantiles1_df = df_onsets.groupby(["quantile1"], as_index=False).mean()
if "quantile2" in df_onsets.keys():
    quantiles2_df = df_onsets.groupby(["quantile2"], as_index=False).mean()
quantilessz_df = df_onsets.groupby(["quantile_sz"], as_index=False).mean()

In [ ]:
# Move from nd2/video-style coordinate system (top left = (0, 0)) to usual plotting coordinate style (bottom left = (0, 0))
quantiles1_df["y_mirrored"]= quantiles1_df.apply(lambda row: -1*row["y"], axis=1)
if "quantile2" in df_onsets.keys():
    quantiles2_df["y_mirrored"]= quantiles2_df.apply(lambda row: -1*row["y"], axis=1)
quantilessz_df["y_mirrored"]= quantilessz_df.apply(lambda row: -1*row["y"], axis=1)

In [ ]:
def get_dx(df, colname="x", quantile_colname="quantile1"):
    max_quantile = df[quantile_colname].max()
    min_quantile = df[quantile_colname].min()
    x1 = df[df[quantile_colname] == max_quantile][colname].values[0]
    x0 = df[df[quantile_colname] == min_quantile][colname].values[0]
    return x1-x0

In [ ]:
def xyToThetaR(vec):
    # given (x, y), return (theta, r)
    return (atan2(vec[1], vec[0]), sqrt(pow(vec[0], 2) + pow(vec[1], 2)))

# descartes coordinates
vec1 = (get_dx(quantiles1_df, "x", "quantile1"), get_dx(quantiles1_df, "y_mirrored", "quantile1"))
if "quantiles2_df" in locals():
    vec2 = ( get_dx(quantiles2_df, "x", "quantile2"), get_dx(quantiles2_df, "y_mirrored", "quantile2") )
vecsz = (get_dx(quantilessz_df, "x", "quantile_sz"), get_dx(quantilessz_df, "y_mirrored", "quantile_sz"))
# polar coordinates as (theta, r)
arrows = {}  # sz, sd1, sd2 order
vec1pol = xyToThetaR(vec1)
arrows["sd1"] = (vec1pol[0], 2)  # categorical length, sz > sd1 > sd2

if "quantiles2_df" in locals():
    vec2pol = xyToThetaR(vec2)
    arrows["sd2"] = (vec2pol[0], 1)
vecszpol = xyToThetaR(vecsz)
arrows["sz"] = ( vecszpol[0], 3)


In [ ]:
event_types = [ "sd2", "sd1", "sz"]
colors = {"sd2":"lightblue", "sd1":"blue", "sz":"darkblue"}
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='polar')
ax.set_ylim(0, 3)
ax.set_title(f"{os.path.split(onsets_h5_fpath)[-1]}", fontsize=20)
for event_type in event_types:
    ax.annotate('', xy=arrows[event_type], xytext=(0, 0),
                arrowprops=dict(facecolor=colors[event_type], edgecolor='none', width=4, headwidth=20, alpha=0.9))
ax.set_rgrids(np.linspace(0, 3, num=len(event_types)+1))
ax.set_yticklabels([""] + event_types)

In [ ]:
f = plt.figure(figsize=(18,12))



ax1 = plt.subplot(231)
ax1.set_title(f"sz", fontsize=20)
sns.heatmap(grid_datasz, annot=False, linewidths=.5, ax=ax1)

ax2= plt.subplot(232)
ax2.set_title(f"sd1", fontsize=20)
sns.heatmap(grid_data1, annot=False, linewidths=.5, ax=ax2)

ax3 = plt.subplot(233)
ax3.set_title(f"sd2", fontsize=20)
sns.heatmap(grid_data2, annot=False, linewidths=.5, ax=ax3)


event_types = [ "sz", "sd1", "sd2"]
colors = {"sd2":"lightblue", "sd1":"blue", "sz":"darkblue"}

i_plot = 234
for i_event, event_type in enumerate(event_types):
    ax = plt.subplot(i_plot, projection='polar')
    ax.set_ylim(0, 3)
    ax.annotate('', xy=arrows[event_type], xytext=(0, 0),arrowprops=dict(facecolor=colors[event_type], edgecolor='none', width=4, headwidth=8, alpha=0.9))
    ax.set_rgrids(np.linspace(0, 3, num=len(event_types)+1))
    ax.set_yticklabels(["", "sd2", "sd1", "sz"])
    #ax.invert_yaxis()
    #ax.invert_xaxis()
    i_plot += 1
plt.show()